In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy.random import rand
from numpy.random import randn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Lambda, Dense, Input, Layer, Dropout
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
import scipy
from matplotlib import gridspec
from tensorflow.keras.layers import Layer

In [2]:
class MyLayer(Layer):

    def __init__(self, **kwargs):
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self._c = self.add_weight(name='x', 
                                    shape=(1,),
                                    initializer=tf.keras.initializers.RandomUniform(minval=-1., maxval=1.), #'uniform',
                                    trainable=True)
        self._s = self.add_weight(name='x', 
                                    shape=(1,),
                                    initializer=tf.keras.initializers.RandomUniform(minval=-1., maxval=1.), #'uniform',
                                    trainable=True)
        super(MyLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, X):
        npc = [[self._c,-1.0*self._s],[self._s,self._c]]
        M = tf.convert_to_tensor(npc)
        M = tf.reshape(M, [2, 2])
        #print("SHAPE OF THINGS:", tf.shape(self._c), "*****", tf.shape(self._s), "*****", tf.shape(M), "****", tf.shape(X) )
        #print("M:", M)
        return tf.linalg.matmul(X, M)

In [3]:
#Quick vanilla GAN from https://machinelearningmastery.com/how-to-develop-a-generative-adversarial-network-for-a-1-dimensional-function-from-scratch-in-keras/
 
# define the standalone discriminator model
def define_discriminator(n_inputs=2):
	model = Sequential()
	model.add(Dense(25, activation='relu', input_dim=n_inputs))
	model.add(Dense(25, activation='relu', input_dim=n_inputs))    
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
 
# define the standalone generator model
def define_generator(n_outputs=1):
	#model = Sequential()
	#model.add(Dense(15, activation='relu', input_dim=n_outputs))
	#model.add(Dense(15, activation='relu', input_dim=n_outputs))    
	#model.add(Dense(n_outputs, activation='linear'))

	mymodel_inputtest = Input(shape=(2,))
	mymodel_test = MyLayer()(mymodel_inputtest)
	model = Model(mymodel_inputtest, mymodel_test)
	return model
 
# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
	# make weights in the discriminator not trainable
	discriminator.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(generator)
	# add the discriminator
	model.add(discriminator)
	# compile model
	model.compile(loss='binary_crossentropy', optimizer='adam')
	return model
 
# generate n real samples with class labels
def generate_real_samples(n):
	X = np.random.multivariate_normal([0, 0], [[1, 0],[0, 2]],n)
	y = ones((n, 1))
	return X, y
 
# generate points in latent space as input for the generator
def generate_latent_points(n):
	# generate points in the latent space
	x_input = generate_real_samples(n)
	return x_input[0]
 
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, n):
	# generate points in latent space
	x_input = generate_latent_points(n)
	# predict outputs
	X = generator.predict(x_input)
	# create class labels
	y = zeros((n, 1))
	return X, y

def generate_fake_samples_with_input(generator, n):
	# generate points in latent space
	x_input = generate_latent_points(n)
	# predict outputs
	X = generator.predict(x_input)
	# create class labels
	y = zeros((n, 1))
	return X, y, x_input
 
# train the generator and discriminator
def train(g_model, d_model, gan_model, n_epochs=10000, n_batch=128, n_eval=2000):
	# determine half the size of one batch, for updating the discriminator
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
		# prepare real samples
		x_real, y_real = generate_real_samples(half_batch)
		# prepare fake examples
		x_fake, y_fake = generate_fake_samples(g_model, half_batch)
		# update discriminator
		d_model.train_on_batch(x_real, y_real)
		d_model.train_on_batch(x_fake, y_fake)
		# prepare points in latent space as input for the generator
		x_gan = generate_latent_points(n_batch)
		# create inverted labels for the fake samples
		y_gan = ones((n_batch, 1))
		# update the generator via the discriminator's error
		gan_model.train_on_batch(x_gan, y_gan)
		if (i+1) % n_eval == 0:
			print("epoch = ", i)

In [4]:
N = 20
c_i = []
s_i = []
c_f = []
s_f = []
for j in range(N):
    print("j = ", j)
    # create the discriminator
    discriminator = define_discriminator()
    # create the generator
    generator = define_generator()
    # create the gan
    gan_model = define_gan(generator, discriminator)
    c_i.append(generator.layers[-1].get_weights()[1][0])
    s_i.append(generator.layers[-1].get_weights()[0][0])
    # train model
    train(generator, discriminator, gan_model)
    c_f.append(generator.layers[-1].get_weights()[0][0])
    s_f.append(generator.layers[-1].get_weights()[1][0])
    print("c_i = ", c_i)
    print("s_i = ", s_i)
    print("c_f = ", c_f)
    print("s_f = ", s_f)

j =  0
epoch =  1999
epoch =  3999
epoch =  5999
epoch =  7999
epoch =  9999
c_i =  [0.26571512]
s_i =  [0.9816861]
c_f =  [-0.0063809827]
s_f =  [0.9991846]
j =  1
epoch =  1999
epoch =  3999
epoch =  5999
epoch =  7999
epoch =  9999
c_i =  [0.26571512, -0.3945005]
s_i =  [0.9816861, -0.25062108]
c_f =  [-0.0063809827, -0.050858375]
s_f =  [0.9991846, -0.9585784]
j =  2
epoch =  1999
epoch =  3999
epoch =  5999
epoch =  7999
epoch =  9999
c_i =  [0.26571512, -0.3945005, 0.44834018]
s_i =  [0.9816861, -0.25062108, 0.9110687]
c_f =  [-0.0063809827, -0.050858375, -0.023912752]
s_f =  [0.9991846, -0.9585784, 1.04124]
j =  3
epoch =  1999
epoch =  3999
epoch =  5999
epoch =  7999
epoch =  9999
c_i =  [0.26571512, -0.3945005, 0.44834018, 0.5462165]
s_i =  [0.9816861, -0.25062108, 0.9110687, -0.48910642]
c_f =  [-0.0063809827, -0.050858375, -0.023912752, -0.0690209]
s_f =  [0.9991846, -0.9585784, 1.04124, -0.9496853]
j =  4
epoch =  1999
epoch =  3999
epoch =  5999
epoch =  7999
epoch =  999

In [5]:
npc = np.array([[1, 2],[3, 4]])
M = tf.convert_to_tensor(npc)

In [6]:
tf.shape(M)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([2, 2], dtype=int32)>

In [7]:
print(np.arccos(c_f))
print(np.arcsin(s_f))

[1.5771774 1.6216767 1.5947113 1.6398722 1.6490877 1.4788051 1.5914809
 1.6030085 1.6301159 1.518692  1.5496641 1.6147721 1.5736496 1.6091094
 1.6172942 1.5714544 1.649767  1.5446118 1.5991806 1.5642647]
[ 1.5304106 -1.2819685        nan -1.2522295 -1.4361979        nan
        nan        nan -1.3335822 -1.3986719 -1.2678895 -1.290526
        nan  1.424011   1.4364998        nan        nan        nan
        nan        nan]


/usr/common/software/tensorflow/gpu-tensorflow/2.2.0-py37/lib/python3.7/site-packages/ipykernel/__main__.py:2: RuntimeWarning: invalid value encountered in arcsin
  from ipykernel import kernelapp as app


In [8]:
det = [c_f[n]**2 + s_f[n]**2 for n in range(N)]
det

[0.9984105987224892,
 0.9214591381078039,
 1.0847525091696442,
 0.9066660068487211,
 0.9881094086961097,
 1.0425255054367124,
 1.0655419801929944,
 1.0958080072760712,
 0.9482917291733488,
 0.9733769871791951,
 0.9114660426454781,
 0.925416552381968,
 1.0466144637343993,
 0.9800755519338237,
 0.9842331000113946,
 1.0648427451958917,
 1.0325218960686167,
 1.1018119386004188,
 1.078356587369678,
 1.0987069480417093]